In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sales = pd.read_csv('../data/read_summarized_sales_clean.csv')
cci = pd.read_csv('../data/read_cci_clean.csv', skiprows=1)
gdp = pd.read_csv('../data/read_gdp_sector_clean.csv')
rates = pd.read_csv('../data/read_fed_funds_rate_clean.csv')
jobs = pd.read_csv('../data/read_nonfarm_jobs_clean.csv')
fuel = pd.read_csv('../data/read_jet_fuel_clean.csv')
unemployment = pd.read_csv('../data/read_unemployment_rate_clean.csv')

In [3]:
sales.head()

,Year,Month,Customer,S_Cons_Order_Class,Market,Total Rev,Order Count
0,2019,Jan,"Abbott, Shannon",At-Once,Health/Wel,245.00,1
1,2019,Jan,"Ahuja, Minh",At-Once,Org,2125.00,1
2,2019,Jan,"al-Ahmad, Nadiyya",At-Once,General,1387.04,2
3,2019,Jan,"al-Akhter, Lubaaba",At-Once,Local,1089.10,2
4,2019,Jan,"al-Assaf, Zuhaira",At-Once,Local,1853.23,1


In [4]:
cci.head()

,Category,Year,Month,United States
0,1/1/2021,2021,Jan,99.25391
1,2/1/2021,2021,Feb,99.39012
2,3/1/2021,2021,Mar,99.74757
3,4/1/2021,2021,Apr,99.98598
4,5/1/2021,2021,May,99.92869


In [5]:
gdp.head()

,Category,GDP,Year,Qtr
0,"Agriculture, forestry, fishing, and hunting",196.4,2021,Q1
1,Mining,279.9,2021,Q1
2,Utilities,386.6,2021,Q1
3,Construction,995.0,2021,Q1
4,Manufacturing,2296.3,2021,Q1


In [11]:
rates.head()

,Obs_Date,Year,Month,Fund_Rate
0,2021-01-01,2021,Jan,0.09
1,2021-02-01,2021,Feb,0.08
2,2021-03-01,2021,Mar,0.07
3,2021-04-01,2021,Apr,0.07
4,2021-05-01,2021,May,0.06


In [13]:
jobs.head()

,Release Date,Actual,Forecast,Previous,Year,Month
0,"Jan 08, 2025 (Dec)",122K,139K,146K,2024,Dec
1,"Dec 04, 2024 (Nov)",146K,166K,184K,2024,Nov
2,"Oct 30, 2024 (Oct)",233K,110K,159K,2024,Oct
3,"Oct 02, 2024 (Sep)",143K,124K,103K,2024,Sep
4,"Sep 05, 2024 (Aug)",99K,144K,111K,2024,Aug


In [15]:
fuel.head()

,Year,Month,Price/Gallon
0,2021,Jan,1.420
1,2022,Jan,2.446
2,2023,Jan,3.542
3,2024,Jan,2.585
4,2021,Feb,1.601


In [17]:
unemployment.head()

,Period,Year,Month,Value
0,M01,2021,Jan,6.4
1,M02,2021,Feb,6.2
2,M03,2021,Mar,6.1
3,M04,2021,Apr,6.1
4,M05,2021,May,5.8
